In [2]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

### Specify input and output

In [9]:
# Specify input and output for the data with BESS
path_input = os.path.join("results", f"load_with_BESS.csv")
path_output = os.path.join("results", f"load_with_BESS_15min.csv")
with_BESS = True

# path_input = os.path.join("data", f"sample_data.csv")
# path_output = os.path.join("results", f"load_without_BESS_15min.csv")
# with_BESS = False

# Change the start and end dates based on the data file
myd_start = "2022-05-31"
myd_end = "2022-06-01"

### Load data with intervals of 1 second

In [10]:
data = pd.read_csv(path_input)

dts = np.array(data['datetime'])
solar_loads = np.array(data['solar_load'])
building_loads = np.array(data['building_load'])
if with_BESS:
    BESS_outputs = np.array(data['BESS_output'])
else:
    BESS_outputs = np.zeros(len(data))
    
dt_start = datetime.strptime(f"{myd_start} 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_end = datetime.strptime(f"{myd_end} 23:59:59", "%Y-%m-%d %H:%M:%S")
billing_interval = timedelta(minutes = 15)

### Get 15-min loads

In [11]:
time_window = [dt_start, dt_start + billing_interval]

solar_load_window = []
building_load_window = []
BESS_output_window = []

solar_loads_15min = []
building_loads_15min = []
BESS_outputs_15min = []
time_points_15min = []

N = len(dts)

for i in range(N):
    dt = datetime.strptime(dts[i], "%Y%m%d-%H%M%S")
    if i+1 < N:
        dt_next = datetime.strptime(dts[i+1], "%Y%m%d-%H%M%S")
    else:
        dt_next = dt_end + timedelta(seconds = 1)
    
    # If current time within the specified 15-min time window, record the values
    if dt >= time_window[0] and dt < time_window[1]:
        solar_load_window.append(solar_loads[i])
        building_load_window.append(building_loads[i])
        BESS_output_window.append(BESS_outputs[i])
    
    # If current time run out of the window, calculate the mean of the previous window and update the window
    if dt_next >= time_window[1]:
        time_window_0_str = datetime.strftime(time_window[0], "%Y%m%d-%H%M%S")
        time_points_15min.append(time_window_0_str)
        
        # If accidently we don't have any data for 15 minutes or longer, we still proceed but a check on the data is necessary
        if len(solar_load_window) == 0:
            time_window[0] = time_window[1]
            time_window[1] += billing_interval
            print("Check data")
            continue
            
        solar_load = np.mean(solar_load_window)
        building_load = np.mean(building_load_window)
        BESS_output = np.mean(BESS_output_window)
        
        solar_loads_15min.append(solar_load)
        building_loads_15min.append(building_load)
        BESS_outputs_15min.append(BESS_output)
        
        # Update time window and data bins
        time_window[0] = time_window[1]
        time_window[1] += billing_interval
        solar_load_window = []
        building_load_window = []
        BESS_output_window = []
    
    if i % 10000 == 0:
        print(i, dt)
print("Finished")

0 2022-05-31 00:00:00
10000 2022-05-31 02:47:52
20000 2022-05-31 05:35:35
30000 2022-05-31 08:23:16
40000 2022-05-31 11:11:10
50000 2022-05-31 13:59:04
60000 2022-05-31 16:46:50
70000 2022-05-31 19:34:45
80000 2022-05-31 22:22:26
90000 2022-06-01 01:28:03
100000 2022-06-01 04:15:55
110000 2022-06-01 07:03:47
120000 2022-06-01 09:51:35
130000 2022-06-01 12:39:22
140000 2022-06-01 15:27:11
150000 2022-06-01 18:15:08
160000 2022-06-01 21:03:01
170000 2022-06-01 23:50:57
Finished


### Export 15-min loads

In [12]:
info_export = {'datetime': time_points_15min,\
               'solar_load': solar_loads_15min,\
               'building_load': building_loads_15min,\
               'BESS_output': BESS_outputs_15min}
df = pd.DataFrame(info_export)
df.to_csv(path_output)